In [1]:
import pandas as pd 
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.preprocessing import StandardScaler,  OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv("data/train.csv")
X=df.drop(columns=["id","Calories"])
y=df['Calories']
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_features = [col for col in X.columns if col not in ["Sex"]]
categorical_features = ["Sex"]

# Create a ColumnTransformer to apply different preprocessing strategies
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(), categorical_features),
    ],
).fit(X_train)

poly = PolynomialFeatures(3)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

poly.fit(X_train)

X_train = poly.transform(X_train)
X_test = poly.transform(X_test)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Step 5: Define parameters for the LightGBM model
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.2,
    'num_leaves': 63,
    'n_estimators': 100,
    'random_seed': 42,
}

# Step 6: Train the model
model = lgb.train(params, train_data, 
                 valid_sets=[test_data],
                 num_boost_round=200)

# Step 7: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 8: Evaluate the model's performance
msle = mean_squared_log_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {msle}")
print(f"R-squared Score: {r2}")

feature_importance = model.feature_importance(importance_type='gain')
sorted_indices = np.argsort(feature_importance)[::-1] 

In [ ]:
import optuna
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Define the Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'random_seed': 42
    }

    # Train the model
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test)

    model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=200)

    # Make predictions
    y_pred = model.predict(X_test)

    # Return RMSE for optimization
    return mean_squared_error(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Run 50 trials

# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# Train the final model with optimized hyperparameters
model_optimized = lgb.train(best_params, lgb.Dataset(X_train, label=y_train),
                            valid_sets=[lgb.Dataset(X_test, label=y_test)], num_boost_round=200)

# Make final predictions
y_pred_final = model_optimized.predict(X_test)

# Evaluate final model
mse_final = mean_squared_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print(f"Optimized Mean Squared Error: {mse_final}")
print(f"Optimized R-squared Score: {r2_final}")

In [ ]:
X_submission = pd.read_csv("data/test.csv")
out=X_submission[["id"]]
X_submission = preprocessor.transform(X_submission.drop(columns=["id"]))
X_submission = poly.transform(X_submission)
y_out = model_optimized.predict(X_submission)
out["Calories"] = y_out
out.to_csv("data/submission.csv", index=False)


In [ ]:
np.log(y)

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor


from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


df = pd.read_csv("data/train.csv")


# Weight_per_Age
df['Weight_per_Age'] = df['Weight'] / (df['Age'] + 1)

# HeartRate per Weight
df['HeartRate_per_kg'] = df['Heart_Rate'] / df['Weight']

# Duration Per Age
df['Duration_per_age'] = df['Duration'] / (df['Age'] + 1)

# Duration * Heart Rate
df['Duration_heart_rate']=df['Duration']*df['Heart_Rate']

# Intensity
df['Duration_per_weight']=df['Duration']/df['Weight']

# All Durations add and multi
df['duration_sum']=df['Duration_per_weight']+df['Duration_heart_rate']+df['Duration_per_age']
df['duration_multi']=df['Duration_per_weight']*df['Duration_heart_rate']*df['Duration_per_age']

# Creating new column 'BMI'
df['BMI']=df['Weight']/(df['Height'] ** 2)
df['BMI']=df['BMI'].round(2)




X = df.drop(columns=["id", "Calories"])

y = np.log(df['Calories'])


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



numerical_features = [col for col in X.columns if col not in ["Sex"]]

categorical_features = ["Sex"]


# Create a ColumnTransformer to apply different preprocessing strategies

preprocessor = ColumnTransformer(

    transformers=[

        ("num", StandardScaler(), numerical_features),

        ("cat", OneHotEncoder(), categorical_features),

    ],

).fit(X_train)


poly = PolynomialFeatures(2, interaction_only=True)


X_train = preprocessor.transform(X_train)

X_test = preprocessor.transform(X_test)


#poly.fit(X_train)


#X_train = poly.transform(X_train)

#X_test = poly.transform(X_test)


#pca = PCA(n_components=0.95)  # Retain 95% variance
#X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)


'''
# Define Autoencoder
input_dim = X_train.shape[1]
latent_dim = 35  # Dimensionality of compressed features

input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(latent_dim, activation='relu')(encoded)

decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train Autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32,
                validation_data=(X_test, X_test), verbose=1)

# Extract compressed features
encoder = Model(inputs=input_layer, outputs=encoded)
X_train = encoder.predict(X_train)
X_test = encoder.predict(X_test)
'''

# Define base models

base_models = [

    ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.061849546072614363, max_depth=5)),

    ('lgbm', LGBMRegressor(n_estimators=3000, learning_rate=0.021849546072614363, max_depth=16)),

    ('cat', CatBoostRegressor(learning_rate=0.06758463422, max_depth=11, iterations =370, verbose=0)),
    
    ('rf', RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=42))

]


# Define Neural Network meta-model
#meta_model = MLPRegressor(hidden_layer_sizes=(
#    128, 32), activation='relu', solver='adam', max_iter=500, random_state=42)


# Define meta-model (aggregator)
meta_model = Ridge()


# Build Stacking Ensemble

stacking_ensemble = StackingRegressor(
    estimators=base_models, final_estimator=meta_model)

stacking_ensemble.fit(X_train, y_train)


# Predictions

y_pred = stacking_ensemble.predict(X_test)
y_pred = np.exp(y_pred)

rmsle_score = mean_squared_log_error(np.exp(y_test), y_pred)
print(f"Optimized Stacking Ensemble RMSLE: {rmsle_score:.4f}")
r2 = r2_score(np.exp(y_test), y_pred)
print(f"R-squared Score: {r2}")

In [ ]:
X_submission = pd.read_csv("data/test.csv")

# Weight_per_Age
X_submission['Weight_per_Age'] = X_submission['Weight'] / (X_submission['Age'] + 1)

# HeartRate per Weight
X_submission['HeartRate_per_kg'] = X_submission['Heart_Rate'] / X_submission['Weight']

# Duration Per Age
X_submission['Duration_per_age'] = X_submission['Duration'] / (X_submission['Age'] + 1)

# Duration * Heart Rate
X_submission['Duration_heart_rate']=X_submission['Duration']*X_submission['Heart_Rate']

# Intensity
X_submission['Duration_per_weight']=X_submission['Duration']/X_submission['Weight']

# All Durations add and multi
X_submission['duration_sum']=X_submission['Duration_per_weight']+X_submission['Duration_heart_rate']+X_submission['Duration_per_age']
X_submission['duration_multi']=X_submission['Duration_per_weight']*X_submission['Duration_heart_rate']*X_submission['Duration_per_age']

# Creating new column 'BMI'
X_submission['BMI']=X_submission['Weight']/(X_submission['Height'] ** 2)
X_submission['BMI']=X_submission['BMI'].round(2)


out=X_submission[["id"]]
X_submission = preprocessor.transform(X_submission.drop(columns=["id"]))
X_submission = poly.transform(X_submission)
y_out =  stacking_ensemble.predict(X_submission)
out["Calories"] = y_out
out.to_csv("data/submission.csv", index=False)

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from bayes_opt import BayesianOptimization
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score


from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


df = pd.read_csv("data/train.csv")


# Weight_per_Age
df['Weight_per_Age'] = df['Weight'] / (df['Age'] + 1)

# HeartRate per Weight
df['HeartRate_per_kg'] = df['Heart_Rate'] / df['Weight']

# Duration Per Age
df['Duration_per_age'] = df['Duration'] / (df['Age'] + 1)

# Duration * Heart Rate
df['Duration_heart_rate']=df['Duration']*df['Heart_Rate']

# Intensity
df['Duration_per_weight']=df['Duration']/df['Weight']

# All Durations add and multi
df['duration_sum']=df['Duration_per_weight']+df['Duration_heart_rate']+df['Duration_per_age']
df['duration_multi']=df['Duration_per_weight']*df['Duration_heart_rate']*df['Duration_per_age']

# Creating new column 'BMI'
df['BMI']=df['Weight']/(df['Height'] ** 2)
df['BMI']=df['BMI'].round(2)




X = df.drop(columns=["id", "Calories"])

y = np.log(df['Calories'])


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



numerical_features = [col for col in X.columns if col not in ["Sex"]]

categorical_features = ["Sex"]


# Create a ColumnTransformer to apply different preprocessing strategies

preprocessor = ColumnTransformer(

    transformers=[

        ("num", StandardScaler(), numerical_features),

        ("cat", OneHotEncoder(), categorical_features),

    ],

).fit(X_train)


poly = PolynomialFeatures(2, interaction_only=True)


X_train = preprocessor.transform(X_train)

X_test = preprocessor.transform(X_test)


# Define base models

base_models = [

    ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.061849546072614363, max_depth=5)),

    ('lgbm', LGBMRegressor(n_estimators=3000, learning_rate=0.021849546072614363, max_depth=16)),

    ('cat', CatBoostRegressor(learning_rate=0.06758463422, max_depth=11, iterations =370, verbose=0))

]


def mlp_evaluate(hidden_layer_1, hidden_layer_2, alpha):
    meta_model = MLPRegressor(
        hidden_layer_sizes=(int(hidden_layer_1), int(hidden_layer_2)),
        activation='relu',
        solver='adam',
        alpha=alpha,
        max_iter=500,
        random_state=42
    )
    
    stacking_ensemble = StackingRegressor(
        estimators=base_models, final_estimator=meta_model
    )
    
    scores = cross_val_score(stacking_ensemble, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    
    return np.mean(scores)

# Define parameter bounds for Bayesian Optimization
param_bounds = {
    'hidden_layer_1': (16, 256),  # Neurons in first layer
    'hidden_layer_2': (8, 128),  # Neurons in second layer
    'alpha': (0.0001, 0.1)        # Regularization strength
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(f=mlp_evaluate, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=10)

# Best parameters found
print("Best parameters for MLPRegressor:", optimizer.max)




In [ ]:
best_params = optimizer.max['params']
optimized_meta_model = MLPRegressor(
    hidden_layer_sizes=(int(best_params['hidden_layer_1']), int(best_params['hidden_layer_2'])),
    activation='relu',
    solver='adam',
    alpha=best_params['alpha'],
    max_iter=500,
    random_state=42
)

# Rebuild the stacking ensemble with optimized MLPRegressor
optimized_stacking_ensemble = StackingRegressor(
    estimators=base_models, final_estimator=optimized_meta_model
)


optimized_stacking_ensemble.fit(X_train, y_train)

# Make predictions
y_pred = optimized_stacking_ensemble.predict(X_test)
y_pred = np.exp(y_pred)

rmsle_score = mean_squared_log_error(np.exp(y_test), y_pred)
print(f"Optimized Stacking Ensemble RMSLE: {rmsle_score:.4f}")
r2 = r2_score(np.exp(y_test), y_pred)
print(f"R-squared Score: {r2}")

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.linear_model import Ridge, ElasticNet, Lasso
from sklearn.metrics import make_scorer
from sklearn.base import BaseEstimator, RegressorMixin, clone
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
import gc
from scipy.stats import uniform, randint
import time

# Custom RMSLE scorer
def rmsle(y_true, y_pred):
    """Calculate Root Mean Squared Logarithmic Error"""
    y_pred = np.maximum(y_pred, 1e-5)
    y_true = np.maximum(y_true, 1e-5)
    return np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

class GPUAcceleratedStackingEnsemble(BaseEstimator, RegressorMixin):
    """GPU-accelerated stacking ensemble that efficiently utilizes GPU resources"""
    def __init__(self, meta_model=None, n_folds=5, random_state=42, gpu_id=0):
        self.meta_model = meta_model if meta_model else ElasticNet(random_state=random_state)
        self.n_folds = n_folds
        self.random_state = random_state
        self.gpu_id = gpu_id
        
        # Will be initialized later
        self.xgb_model = None
        self.lgb_model = None
        self.ctb_model = None
        
        self.base_models = []
        self.base_models_trained = []
    
    def _initialize_models(self):
        # Initialize with GPU settings
        self.xgb_model = xgb.XGBRegressor(
            objective='reg:squaredlogerror', 
            random_state=self.random_state,
            tree_method='gpu_hist',  # GPU acceleration
            gpu_id=self.gpu_id
        )
        
        self.lgb_model = lgb.LGBMRegressor(
            objective='regression', 
            random_state=self.random_state,
            device='gpu',  # GPU acceleration
            gpu_platform_id=0,
            gpu_device_id=self.gpu_id
        )
        
        self.ctb_model = ctb.CatBoostRegressor(
            loss_function='RMSE', 
            random_state=self.random_state,
            verbose=0,
            task_type='GPU',  # GPU acceleration
            devices=f'{self.gpu_id}'
        )
        
        self.base_models = [self.xgb_model, self.lgb_model, self.ctb_model]
    
    def fit(self, X, y):
        if len(self.base_models) == 0:
            self._initialize_models()
            
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            start_time = time.time()
            print(f"Training model {i+1}/{len(self.base_models)} ({model.__class__.__name__})")
            
            # Generate out-of-fold predictions
            for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
                X_train, X_val = X[train_idx], X[val_idx]
                y_train, y_val = y[train_idx], y[val_idx]

                
                clone_model = clone(model)
                clone_model.fit(X_train, y_train)
                meta_features[val_idx, i] = clone_model.predict(X_val)
                
                del clone_model
                gc.collect()
            
            # Fit on full dataset
            model_copy = clone(model)
            model_copy.fit(X, y)
            self.base_models_trained.append(model_copy)
            
            print(f"Model {i+1} training completed in {time.time() - start_time:.2f} seconds")
            gc.collect()
        
        # Train meta-model
        self.meta_model.fit(meta_features, y)
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            model.predict(X) for model in self.base_models_trained
        ])
        return self.meta_model.predict(meta_features)

def gpu_accelerated_tuning(X_train, y_train, X_test=None, y_test=None, cv=5, gpu_id=0):
    """
    GPU-accelerated hyperparameter tuning with automated batch size selection
    and memory management for optimal performance
    """
    best_models = []
    
    # Calculate appropriate batch sizes based on dataset size and available GPU memory
    # These are examples and should be adjusted based on your GPU memory
    data_size_mb = X_train.nbytes / (1024 * 1024)
    print(f"Dataset size: {data_size_mb:.2f} MB")
    
    # Parameter distributions
    xgb_param_dist = {
        'max_depth': randint(5, 30),
        'learning_rate': uniform(0.01, 0.3),
        'n_estimators': randint(500, 3000),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'reg_alpha': uniform(0, 1),
        'reg_lambda': uniform(0, 5),
        # GPU-specific parameters
        'max_bin': randint(128, 512),  # Controls GPU memory usage
        'gpu_id': [gpu_id]
    }
    
    lgb_param_dist = {
        'num_leaves': randint(20, 100),
        'learning_rate': uniform(0.01, 0.3),
        'n_estimators': randint(500, 3000),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'reg_alpha': uniform(0, 1),
        'reg_lambda': uniform(0, 1),
        # GPU-specific parameters
        'device': ['gpu'],
        'gpu_platform_id': [0],
        'gpu_device_id': [gpu_id],
        'max_bin': randint(128, 255)  # Controls GPU memory usage
    }
    
    ctb_param_dist = {
        'depth': randint(8, 16),
        'learning_rate': uniform(0.01, 0.5),
        'iterations': randint(500, 3000),
        'l2_leaf_reg': uniform(4, 16),
        # GPU-specific parameters
        'task_type': ['GPU'],
        'devices': [f'{gpu_id}'],
        'gpu_ram_part': uniform(0.3, 0.7)  # Portion of GPU memory to use
    }
    
    # Tune XGBoost with GPU
    print("Tuning XGBoost with GPU acceleration...")
    start_time = time.time()
    xgb_model = xgb.XGBRegressor(
        objective='reg:squaredlogerror', 
        random_state=42,
        tree_method='gpu_hist',  # GPU algorithm
        gpu_id=gpu_id
    )
    
    xgb_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=xgb_param_dist,
        n_iter=20,
        scoring=rmsle_scorer,
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=1  # Use 1 for GPU to avoid conflicts
    )
    
    xgb_search.fit(X_train, y_train)
    print(f"Best XGBoost RMSLE: {-xgb_search.best_score_:.5f}")
    print(f"Best XGBoost params: {xgb_search.best_params_}")
    print(f"XGBoost tuning completed in {time.time() - start_time:.2f} seconds")
    best_models.append(('xgb', xgb_search.best_estimator_))
    
    best_xgb = clone(xgb_search.best_estimator_)
    del xgb_search, xgb_model
    gc.collect()
    
    # Tune LightGBM with GPU
    print("\nTuning LightGBM with GPU acceleration...")
    start_time = time.time()
    lgb_model = lgb.LGBMRegressor(
        objective='regression', 
        random_state=42,
        device='gpu',
        gpu_platform_id=0,
        gpu_device_id=gpu_id,
        verbose = 0
    )
    
    lgb_search = RandomizedSearchCV(
        estimator=lgb_model,
        param_distributions=lgb_param_dist,
        n_iter=20,
        scoring=rmsle_scorer,
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=1  # Use 1 for GPU to avoid conflicts
    )
    
    lgb_search.fit(X_train, y_train)
    print(f"Best LightGBM RMSLE: {-lgb_search.best_score_:.5f}")
    print(f"Best LightGBM params: {lgb_search.best_params_}")
    print(f"LightGBM tuning completed in {time.time() - start_time:.2f} seconds")
    best_models.append(('lgb', lgb_search.best_estimator_))
    
    best_lgb = clone(lgb_search.best_estimator_)
    del lgb_search, lgb_model
    gc.collect()
    
    # Tune CatBoost with GPU
    print("\nTuning CatBoost with GPU acceleration...")
    start_time = time.time()
    ctb_model = ctb.CatBoostRegressor(
        loss_function='RMSE', 
        random_state=42, 
        verbose=0,
        task_type='GPU',
        devices=f'{gpu_id}'
    )
    
    ctb_search = RandomizedSearchCV(
        estimator=ctb_model,
        param_distributions=ctb_param_dist,
        n_iter=20,
        scoring=rmsle_scorer,
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=1  # Use 1 for GPU to avoid conflicts
    )
    
    ctb_search.fit(X_train, y_train)
    print(f"Best CatBoost RMSLE: {-ctb_search.best_score_:.5f}")
    print(f"Best CatBoost params: {ctb_search.best_params_}")
    print(f"CatBoost tuning completed in {time.time() - start_time:.2f} seconds")
    best_models.append(('ctb', ctb_search.best_estimator_))
    
    best_ctb = clone(ctb_search.best_estimator_)
    del ctb_search, ctb_model
    gc.collect()
    
    # Now tune the meta-model (CPU-based since these are simple models)
    print("\nTuning Stacking Ensemble...")
    
    meta_models = [
        {
            'name': 'ElasticNet',
            'model': ElasticNet(random_state=42),
            'param_dist': {
                'alpha': uniform(0.0001, 1.0),
                'l1_ratio': uniform(0.1, 0.8)
            }
        },
        {
            'name': 'Ridge',
            'model': Ridge(random_state=42),
            'param_dist': {
                'alpha': uniform(0.0001, 10.0)
            }
        },
        {
            'name': 'Lasso',
            'model': Lasso(random_state=42),
            'param_dist': {
                'alpha': uniform(0.0001, 1.0)
            }
        }
    ]
    
    best_score = float('inf')
    best_meta_model = None
    best_ensemble = None
    
    for meta_info in meta_models:
        print(f"\nTesting {meta_info['name']} as meta-model...")
        start_time = time.time()
        
        stacking = GPUAcceleratedStackingEnsemble(
            meta_model=meta_info['model'],
            n_folds=cv,
            random_state=42,
            gpu_id=gpu_id
        )
        
        stacking._initialize_models()
        stacking.base_models = [best_xgb, best_lgb, best_ctb]
        
        param_dist = {f'meta_model__{k}': v for k, v in meta_info['param_dist'].items()}
        
        search = RandomizedSearchCV(
            estimator=stacking,
            param_distributions=param_dist,
            n_iter=10,
            scoring=rmsle_scorer,
            cv=cv,
            verbose=1,
            random_state=42,
            n_jobs=1  # Use 1 for GPU
        )
        
        search.fit(X_train, y_train)
        current_score = -search.best_score_
        
        print(f"{meta_info['name']} best RMSLE: {current_score:.5f}")
        print(f"{meta_info['name']} best params: {search.best_params_}")
        print(f"{meta_info['name']} tuning completed in {time.time() - start_time:.2f} seconds")
        
        if current_score < best_score:
            best_score = current_score
            best_meta_model = meta_info['name']
            best_ensemble = search.best_estimator_
        
        del search, stacking
        gc.collect()
    
    print(f"\nBest Meta-Model: {best_meta_model}")
    print(f"Best Stacking Ensemble RMSLE: {best_score:.5f}")
    
    # Evaluate on test set if provided
    if X_test is not None and y_test is not None:
        y_pred = best_ensemble.predict(X_test)
        test_rmsle = rmsle(y_test, y_pred)
        print(f"Test RMSLE: {test_rmsle:.5f}")
        
        for name, model in best_models:
            y_pred = model.predict(X_test)
            test_rmsle = rmsle(y_test, y_pred)
            print(f"{name} Test RMSLE: {test_rmsle:.5f}")
    
    return best_ensemble, best_models





In [ ]:
df = pd.read_csv("data/train.csv")


df['Weight_per_Age'] = df['Weight'] / (df['Age'] + 1)

# HeartRate per Weight
df['HeartRate_per_kg'] = df['Heart_Rate'] / df['Weight']

# Duration Per Age
df['Duration_per_age'] = df['Duration'] / (df['Age'] + 1)

# Duration * Heart Rate
df['Duration_heart_rate'] = df['Duration']*df['Heart_Rate']

# Intensity
df['Duration_per_weight'] = df['Duration']/df['Weight']

# All Durations add and multi
df['duration_sum'] = df['Duration_per_weight'] + \
    df['Duration_heart_rate']+df['Duration_per_age']
df['duration_multi'] = df['Duration_per_weight'] * \
    df['Duration_heart_rate']*df['Duration_per_age']

# Creating new column 'BMI'
df['BMI'] = df['Weight']/(df['Height'] ** 2)
df['BMI'] = df['BMI'].round(2)

df['Body_Temp2'] = df['Body_Temp']**2

X = df.drop(columns=["id", "Calories"])

y = df['Calories']

numerical_features = [col for col in X.columns if col not in ["Sex"]]

categorical_features = ["Sex"]
X['Sex'] = X['Sex'].map({'female': 1, 'male': 0})

for col in categorical_features:
    for num_feature in numerical_features:
        X[f'{num_feature}_x_{col}'] = X[num_feature] * X[col]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


numerical_features = [col for col in X.columns if col not in ["Sex"]]

categorical_features = ["Sex"]


# Create a ColumnTransformer to apply different preprocessing strategies

preprocessor = ColumnTransformer(

    transformers=[

        ("num", StandardScaler(), numerical_features),

    ],
    remainder = "passthrough"

).fit(X)



X_train = preprocessor.transform(X_train)

X_test = preprocessor.transform(X_test)


best_model, base_models = gpu_accelerated_tuning(X_train, y_train, X_test, y_test, gpu_id=0)



Dataset size: 141.91 MB
Tuning XGBoost with GPU acceleration...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:44:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\core.py:2676: UserWarning: [00:46:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:46:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree met

In [ ]:
best_model, base_models
